convert notebook to open api?


In [5]:
var path = require('path');
var importer = require('../Core');
var getRpcFromSpec = importer.import('get rpc from spec');
var getPermissions = importer.import('rpc permissions');
var groupPermissions = importer.import('test rpc permissions');
var addAST = importer.import('get all ast properties')
var package = require('../package.json');

// TODO: move this to patterns utility
function getTail(path, ctx) {
    var cur = path.split('.')[0];
    if(!cur) return ctx;
    if(typeof ctx[cur] === 'undefined') ctx[cur] = {};
    return getTail(path.split('.').slice(1).join('.'), ctx[cur]);
}

function getPathId(cell) {
    return path.relative('../', path.dirname(cell.filename)).replace(/\//ig, '.resources.').toLowerCase();
}

function getRpcSpecFromCells(search) {
    var functions = groupPermissions(['Function', 'Available'], search);
    var resources = Object.keys(functions).reduce((obj, f) => {
        var cell = addAST(importer.interpret(f));
        var path = getPathId(cell);
        if(path) path += '.resources.';
        var notebook = cell.notebook.replace(/\.ipynb|\s+/ig, '').toLowerCase();
        var fn = cell.params[0];
        var tail = getTail(`${path}${notebook}`, obj);
        if(typeof tail.methods === 'undefined') tail.methods = {};
        tail.methods[fn] = {
            description: cell.markdown,
            scopes: functions[f],
            id: `${package.name}.${path}${notebook}.${fn}`,
            httpMethod: 'GET',
            path: `?function=${cell.questions[0]}`,
            parameters: cell.params.slice(1).reduce((o, p) => (o[p] = {
                type: 'string',
                location: 'query',
                required: true,
                description: ''
            }, o), {})
        };
        return obj;
    }, {});
    return {
        title: 'Jupyter Ops',
        name: package.name,
        description: package.description,
        protocol: 'rest',
        baseUrl: process.env.FUNCTIONS_URL,
        resources: resources,
        auth: {},
        schemas: {},
        parameters: {}
    }
}

module.exports = getRpcSpecFromCells;


[Function: getRpcFromCells]

test convert notebook api?


In [11]:
var importer = require('../Core');
var getRpcSpecFromCells = importer.import('convert notebook to open api');
var getRpcFromSpec = importer.import('get rpc from spec');
var getEnvironment = importer.import('get environment');
var groupPermissions = importer.import('test get permissions');
getEnvironment('STUDY_LOCAL')

if(typeof $$ !== 'undefined') {
    $$.async()
    var spec = getRpcSpecFromCells('study sauce.ipynb');
    console.log(spec);
    var juypter_ops = getRpcFromSpec(spec);
    console.log(juypter_ops);
//    juypter_ops.rpc.getPermissions({search: null})
    juypter_ops.rpc.getPermissions({search: null})
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb
compiling /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb[8] aka get rpc from spec
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb
compiling /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb[7] aka get environment
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb
compiling /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb[3] aka test get permissions
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb
compiling /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb[2] aka rpc permissions
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb
compiling /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb[1] aka get cell rpc groups
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb
compiling /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb[0] aka rpc group

TypeError: Cannot read property 'getPermissions' of undefined

convert google discovery to documentation?

https://developers.google.com/apis-explorer/#s/discovery/v1/discovery.apis.getRest?api=discovery&version=v1&_h=2&

https://www.googleapis.com/discovery/v1/apis/discovery/v1/rest

https://developers.google.com/apis-explorer/#p/discovery/v1/discovery.apis.list?_h=1&




discover well-known apis?

TODO: figure out how OpenAPI, Swagger RAML do this, Google's is obvious

https://tools.ietf.org/html/rfc5785
